In [29]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
#__author__:yanbin
from pyspark import SparkContext,SparkConf
from operator import add,itemgetter
from pyspark.sql import SparkSession,Row
from pandas import *
from pyspark.sql.functions import *
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import *
import time
import math
class UserCFModel(object):  
    def __init__(self,df,user_pair_sim=None):
        self.df = df
        self.user_pair_sim = user_pair_sim

    def userSimilarity(self):
        # RDD[(aid,(uid,score))] 
        user_item_score = self.df.rdd.map(lambda x:(x[0],[x[1],x[2]]))
        user_items_score = user_item_score.groupByKey().mapValues(dict)
        user_pair_XX_YY = user_items_score.mapValues(lambda f:f[0][0] == f[0][1])
        
        user_score_pair = item_user_score.join(item_user_score)\
                        .map(lambda x:((x[1][0][0],x[1][1][0]),(x[1][0][1],x[1][1][1])))
        user_pair_ALL = user_score_pair.map(lambda f:(f[0], f[1][0] * f[1][1])).reduceByKey(add,300)
        #RDD(uid1,((uid1,uid2,XY),XX))
        user_pair_XY = user_pair_ALL.filter(lambda f:f[0][0] != f[0][1])
        #RDD[(uid1,score11 * score11 + score21 * score21)] 
        user_XX_YY = user_pair_XX_YY.map(lambda f:(f[0][0], f[1]))
        user_XY_XX = user_pair_XY.map(lambda f:(f[0][0], (f[0][0], f[0][1], f[1]))).join(user_XX_YY) 
        #RDD[(uid2,((uid1,uid2,
        #           score11 * score12 + score21 * score22,score11 * score11 + score21 * score21),
        #           score12 * score12 + score22 * score22))] 
        user_XY_XX_YY = user_XY_XX.map(lambda f:(f[1][0][1],(f[1][0][0],f[1][0][1],f[1][0][2],f[1][1]))).join(user_XX_YY)  
        # user_XY_XX_YY中的(uid1,uid2,XY,XX,YY)) 
        # RDD[(uid1,uid2,
        # score11 * score12 + score21 * score22,score11 * score11 + score21 * score21,score12 * score12 + score22 * score22)]       
        user_pair_XY_XX_YY = user_XY_XX_YY.map(lambda f:(f[1][0][0], f[1][0][1], f[1][0][2], f[1][0][3], f[1][1]))  
        # user_pair_XY_XX_YY为(uid1,uid2,XY / math.sqrt(XX * YY)) 
        # RDD[(aid1,aid2,
        # score11 * score12 + score21 * score22 / math.sqrt((score11 * score11 + score21 * score21)*(score12 * score12 + score22 * score22))] 
        user_pair_sim = user_pair_XY_XX_YY.map(lambda f :(f[0], (f[1], f[2] / math.sqrt(f[3] * f[4]))))  
        return user_pair_sim
    

    def train(self):
        user_pair_sim = self.userSimilarity()
        user_pair_sim.cache()  
        self.user_pair_sim=user_pair_sim
    



def recommend(df,user_pair_sim,NN=100,topN =10,Normalization=False):
    def userNN(user_pair_sim,NN=100,Normalization=False):
        user_sim = user_pair_sim.filter(lambda f:f[1][1]>0.05)\
                            .groupByKey()\
                            .mapValues(list)
        if Normalization:
            def norm(x):
                m =  __builtin__.max([i[1] for i in x])
                l = []
                for i in x:
                    l.append((i[0],i[1]/m))
                return l
            user_sim = user_sim.mapValues(lambda x:norm(x))
        user_simNN = user_sim.mapValues(lambda x:sorted(x,key=itemgetter(1),reverse=True)[:NN])\
                            .collectAsMap()
        return user_simNN
    
    def getOrElse(f,user_sim_bd,user_item_score):
        users_sim = user_sim_bd.value.get(f[0][0]) 
        if users_sim is None:
            users_sim = [("0", 0.0)]
        for w in users_sim:
            zs = user_items_score_bd.value.get(w[0])
            if zs is None: zs=[("0",0.0)]
            for z in zs:
                yield ((f[0][0],z[0]),w[1]*z[1])
            
    user_item_score = df.rdd.map(lambda x:((x[0],x[1]),x[2]))
    user_sim_bd = sc.broadcast(userNN(user_pair_sim,NN=NN,Normalization=Normalization))
    user_items_score = df.rdd.map(lambda x:(x[0],(x[1],x[2]))).groupByKey().mapValues(list).collectAsMap()
    user_items_score_bd = sc.broadcast(user_items_score)
#     /* 
#      * 提取item_sim_user_score为((user,item2),sim * score) 
#      * RDD[(user,item2),sim * score] 
#      */  

    user_item_simscore = user_item_score.flatMap(lambda f:getOrElse(f,user_sim_bd,user_items_score_bd))\
                                        .filter(lambda f:f[1]> 0.03)  
#       /*
#      * 聚合user_item_simscore为 (user,（item2,sim1 * score1 + sim2 * score2）)
#      * 假设user观看过两个item,评分分别为score1和score2，item2是与user观看过的两个item相似的item,相似度分别为sim1，sim2 
#      * RDD[(user,item2),sim1 * score1 + sim2 * score2）)] 
#      */  
    user_item_rank = user_item_simscore.reduceByKey(add,1000)  

#     /* 
#      * 过滤用户已看过的item,并对user_item_rank基于user聚合 
#      * RDD[(user,CompactBuffer((item2,rank2）,(item3,rank3)...))] 
#      */  
    user_items_ranks = user_item_rank.subtractByKey(user_item_score)\
                                     .map(lambda f:(f[0][0], (f[0][1], f[1])))\
                                     .groupByKey()  
#     /* 
#      * 对user_items_ranks基于rank降序排序，并提取topN,其中包括用户已观看过的item 
#      * RDD[(user,ArrayBuffer((item2,rank2）,...,(itemN,rankN)))] 
#      */  
    user_items_ranks_desc = user_items_ranks.mapValues(list)\
                            .mapValues(lambda x:sorted(x,key=itemgetter(1),reverse=True)[:topN])
    return user_items_ranks_desc

from Evaluator import Evaluator
def evaluate(evaluator):
    precise = evaluator.precision()
    coverage = evaluator.coverage()
    popularity = evaluator.popularity()
    recall = evaluator.recall()
    return precise,recall,coverage,popularity

if __name__ == "__main__": 
    import os
    PYSPARK_PYTHON = "/usr/bin/python2.7"
    os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
    conf = SparkConf().setAppName("itemCF").setMaster("yarn")
    conf.set('spark.yarn.dist.files',
            'file:/root/hadoop-2.6/spark/python/lib/pyspark.zip,file:/root/hadoop-2.6/spark/python/lib/py4j-0.10.4-src.zip')
    conf.set('spark.executor.cores','30')
    conf.set('spark.executor.memory','95g')
    conf.set('spark.executor.instances','8')
    conf.set('spark.sql.shuffle.partitions','400')
    conf.set('spark.default.parallelism','200')
    conf.set('spark.scheduler.mode','fair')
#     conf.set("spark.shuffle.file.buffer","128k").set("spark.reducer.maxSizeInFlight","96M")
    spark = SparkSession.builder\
        .config(conf=conf) \
        .getOrCreate()
    sc=spark.sparkContext 
    sc.setLogLevel('WARN')
    start = time.time()
    inputPath = 'data/ml-1m/ratings.dat'
    schema = StructType([
            StructField("user", StringType(), True),
            StructField("item", StringType(), True),
            StructField("rating", DoubleType(), True),
            StructField("timestamp", LongType(), True)])
    ratingRdd = sc.textFile(inputPath).map(lambda line:line.split("::"))\
                    .map(lambda x:(x[0],x[1],float(x[2]),long(x[3])))
    ratingDf = spark.createDataFrame(data=ratingRdd,schema=schema)
#     ratingDf,_ = ratingDf.randomSplit([1.0,9.0],seed=40)
    ratingDf = ratingDf.repartition(300)
    ratingDf.printSchema()
    ratingDf.show(5)
    n = ratingDf.count()
#     ratingDf = ratingDf.withColumn('score',col('rating')*0+1).select('user','item','score')    
    print 'total lines: %s' %n
    train,test = ratingDf.randomSplit([4.0,1.0],seed=40)
    train.cache()
    test.cache()
    userCF = UserCFModel(df=train)
    userCF.train()
    for NN in [5,10,20,40,60,80,100,120]:
        recTopN = recommend(train,userCF.user_pair_sim,NN=NN,topN=10,Normalization=False)
        pre = spark.createDataFrame(data=recTopN.flatMapValues(lambda x:x).map(lambda x:(x[0],x[1][0],x[1][1])),
                                    schema=['user','item','rating'])
        evaluator = Evaluator(train,test,pre)
        (precise,recall,coverage,popularity) = evaluate(evaluator)
        end = time.time()
        print ('NN:%3d,  precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f, time: %s s'
                %(NN,precise*100,recall*100,coverage*100,popularity,end-start))
        pre.write.parquet(partitionBy='user',path='data/ml-1m/result/userCF/uscf_'+str(NN),mode='ignore')
        start = time.time()

root
 |-- user: string (nullable = true)
 |-- item: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)

+----+----+------+---------+
|user|item|rating|timestamp|
+----+----+------+---------+
|   4|2028|   5.0|978294230|
|   7|1270|   4.0|978234581|
|  10|   2|   5.0|979168267|
|  10|1654|   5.0|979168346|
|  13|1204|   5.0|978202201|
+----+----+------+---------+
only showing top 5 rows

total lines: 1000209


In [42]:
from scipy.linalg.misc import norm
# user_item_score = test.rdd.map(lambda x:(x[0],[x[1],x[2]]))
# user_items_score = user_item_score.groupByKey().mapValues(dict)
user_pair_XX_YY = user_items_score.map(lambda x:(x[0],norm(x[1].values()),x[1]))
def myDot(x,y):
    c = 0
    for k,v in x.items():
        if k in y:
            c += v*y[k]
    return c
user_pair_sim = user_pair_XX_YY.cartesian(user_pair_XX_YY).filter(lambda x:x[0][0]!=x[1][0])\
                    .map(lambda x:((x[0][0],x[1][0]),myDot(x[0][2],x[1][2])*1.0/(x[0][1]*x[1][1])))

user_pair_sim.collect()
# user_score_pair = item_user_score.join(item_user_score)\
#                 .map(lambda x:((x[1][0][0],x[1][1][0]),(x[1][0][1],x[1][1][1])))
# user_pair_ALL = user_score_pair.map(lambda f:(f[0], f[1][0] * f[1][1])).reduceByKey(add,300)
# #RDD(uid1,((uid1,uid2,XY),XX))
# user_pair_XY = user_pair_ALL.filter(lambda f:f[0][0] != f[0][1])
# #RDD[(uid1,score11 * score11 + score21 * score21)] 
# user_XX_YY = user_pair_XX_YY.map(lambda f:(f[0][0], f[1]))
# user_XY_XX = user_pair_XY.map(lambda f:(f[0][0], (f[0][0], f[0][1], f[1]))).join(user_XX_YY) 
# #RDD[(uid2,((uid1,uid2,
# #           score11 * score12 + score21 * score22,score11 * score11 + score21 * score21),
# #           score12 * score12 + score22 * score22))] 
# user_XY_XX_YY = user_XY_XX.map(lambda f:(f[1][0][1],(f[1][0][0],f[1][0][1],f[1][0][2],f[1][1]))).join(user_XX_YY)  
# # user_XY_XX_YY中的(uid1,uid2,XY,XX,YY)) 
# # RDD[(uid1,uid2,
# # score11 * score12 + score21 * score22,score11 * score11 + score21 * score21,score12 * score12 + score22 * score22)]       
# user_pair_XY_XX_YY = user_XY_XX_YY.map(lambda f:(f[1][0][0], f[1][0][1], f[1][0][2], f[1][0][3], f[1][1]))  
# # user_pair_XY_XX_YY为(uid1,uid2,XY / math.sqrt(XX * YY)) 
# # RDD[(aid1,aid2,
# # score11 * score12 + score21 * score22 / math.sqrt((score11 * score11 + score21 * score21)*(score12 * score12 + score22 * score22))] 
# user_pair_sim = user_pair_XY_XX_YY.map(lambda f :(f[0], (f[1], f[2] / math.sqrt(f[3] * f[4]))))  
# return user_pair_sim

[((u'2121', u'2051'), 0.0),
 ((u'2121', u'5722'), 0.023596896866014697),
 ((u'2121', u'3885'), 0.014691558658335683),
 ((u'2121', u'5829'), 0.08036342149471598),
 ((u'2121', u'5627'), 0.04692570636947007),
 ((u'2121', u'5959'), 0.0),
 ((u'2121', u'861'), 0.0),
 ((u'2121', u'5564'), 0.0),
 ((u'2121', u'2046'), 0.0),
 ((u'2121', u'5612'), 0.0),
 ((u'2121', u'5542'), 0.0),
 ((u'2121', u'5717'), 0.02200842937547574),
 ((u'2121', u'876'), 0.0),
 ((u'2121', u'5634'), 0.0),
 ((u'2121', u'5982'), 0.0),
 ((u'2121', u'5577'), 0.0),
 ((u'2121', u'5599'), 0.0),
 ((u'2121', u'2136'), 0.0),
 ((u'2121', u'5704'), 0.0),
 ((u'2051', u'2121'), 0.0),
 ((u'5722', u'2121'), 0.023596896866014697),
 ((u'2051', u'5722'), 0.06122908023640053),
 ((u'5722', u'2051'), 0.06122908023640053),
 ((u'2051', u'3885'), 0.003176797034643827),
 ((u'2051', u'5829'), 0.0),
 ((u'2051', u'5627'), 0.04960695628494882),
 ((u'2051', u'5959'), 0.08660687083024937),
 ((u'5722', u'3885'), 0.0380379727256645),
 ((u'5722', u'5829'), 0

In [39]:
rdd1 = sc.parallelize([('a',2),('b',4),('c',6),('d',9)])
rdd2 = sc.parallelize([('c',6),('c',7),('d',8),('e',10)])
print 'rdd1',rdd1.collect()
print 'rdd2',rdd2.collect()
# print 'join',rdd1.join(rdd2).collect()
# print 'leftOutjoin',rdd1.leftOuterJoin(rdd2).collect()
# print 'rightOutjoin',rdd1.rightOuterJoin(rdd2).collect()
print 'catesian',rdd1.cartesian(rdd1).collect()

rdd1 [('a', 2), ('b', 4), ('c', 6), ('d', 9)]
rdd2 [('c', 6), ('c', 7), ('d', 8), ('e', 10)]
catesian [(('a', 2), ('a', 2)), (('a', 2), ('b', 4)), (('a', 2), ('c', 6)), (('a', 2), ('d', 9)), (('b', 4), ('a', 2)), (('b', 4), ('b', 4)), (('b', 4), ('c', 6)), (('b', 4), ('d', 9)), (('c', 6), ('a', 2)), (('c', 6), ('b', 4)), (('c', 6), ('c', 6)), (('c', 6), ('d', 9)), (('d', 9), ('a', 2)), (('d', 9), ('b', 4)), (('d', 9), ('c', 6)), (('d', 9), ('d', 9))]


In [ ]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
#__author__:yanbin
from pyspark import SparkContext,SparkConf
from operator import add,itemgetter
from pyspark.sql import SparkSession,Row
from pandas import *
from pyspark.sql.functions import *
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import *
import time
import math
class UserCFModel(object):  
    def __init__(self,df,user_pair_sim=None):
        self.df = df
        self.user_pair_sim = user_pair_sim

    def userSimilarity(self):
        # RDD[(aid,(uid,score))] 
        item_user_score = self.df.rdd.map(lambda x:(x[1],[x[0],x[2]]))
        user_score_pair = item_user_score.join(item_user_score)\
                        .map(lambda x:((x[1][0][0],x[1][1][0]),(x[1][0][1],x[1][1][1])))
        user_pair_ALL = user_score_pair.map(lambda f:(f[0], f[1][0] * f[1][1])).reduceByKey(add,300)
        user_pair_XX_YY = user_pair_ALL.filter(lambda f:f[0][0] == f[0][1])
        #RDD(uid1,((uid1,uid2,XY),XX))
        user_pair_XY = user_pair_ALL.filter(lambda f:f[0][0] != f[0][1])
        #RDD[(uid1,score11 * score11 + score21 * score21)] 
        user_XX_YY = user_pair_XX_YY.map(lambda f:(f[0][0], f[1]))
        user_XY_XX = user_pair_XY.map(lambda f:(f[0][0], (f[0][0], f[0][1], f[1]))).join(user_XX_YY) 
        #RDD[(uid2,((uid1,uid2,
        #           score11 * score12 + score21 * score22,score11 * score11 + score21 * score21),
        #           score12 * score12 + score22 * score22))] 
        user_XY_XX_YY = user_XY_XX.map(lambda f:(f[1][0][1],(f[1][0][0],f[1][0][1],f[1][0][2],f[1][1]))).join(user_XX_YY)  
        # user_XY_XX_YY中的(uid1,uid2,XY,XX,YY)) 
        # RDD[(uid1,uid2,
        # score11 * score12 + score21 * score22,score11 * score11 + score21 * score21,score12 * score12 + score22 * score22)]       
        user_pair_XY_XX_YY = user_XY_XX_YY.map(lambda f:(f[1][0][0], f[1][0][1], f[1][0][2], f[1][0][3], f[1][1]))  
        # user_pair_XY_XX_YY为(uid1,uid2,XY / math.sqrt(XX * YY)) 
        # RDD[(aid1,aid2,
        # score11 * score12 + score21 * score22 / math.sqrt((score11 * score11 + score21 * score21)*(score12 * score12 + score22 * score22))] 
        user_pair_sim = user_pair_XY_XX_YY.map(lambda f :(f[0], (f[1], f[2] / math.sqrt(f[3] * f[4]))))  
        return user_pair_sim
    

    def train(self):
        user_pair_sim = self.userSimilarity()
        user_pair_sim.cache()  
        self.user_pair_sim=user_pair_sim
    



def recommend(df,user_pair_sim,NN=100,topN =10,Normalization=False):
    def userNN(user_pair_sim,NN=100,Normalization=False):
        user_sim = user_pair_sim.filter(lambda f:f[1][1]>0.05)\
                            .groupByKey()\
                            .mapValues(list)
        if Normalization:
            def norm(x):
                m =  __builtin__.max([i[1] for i in x])
                l = []
                for i in x:
                    l.append((i[0],i[1]/m))
                return l
            user_sim = user_sim.mapValues(lambda x:norm(x))
        user_simNN = user_sim.mapValues(lambda x:sorted(x,key=itemgetter(1),reverse=True)[:NN])\
                            .collectAsMap()
        return user_simNN
    
    def getOrElse(f,user_sim_bd,user_item_score):
        users_sim = user_sim_bd.value.get(f[0][0]) 
        if users_sim is None:
            users_sim = [("0", 0.0)]
        for w in users_sim:
            zs = user_items_score_bd.value.get(w[0])
            if zs is None: zs=[("0",0.0)]
            for z in zs:
                yield ((f[0][0],z[0]),w[1]*z[1])
            
    user_item_score = df.rdd.map(lambda x:((x[0],x[1]),x[2]))
    user_sim_bd = sc.broadcast(userNN(user_pair_sim,NN=NN,Normalization=Normalization))
    user_items_score = df.rdd.map(lambda x:(x[0],(x[1],x[2]))).groupByKey().mapValues(list).collectAsMap()
    user_items_score_bd = sc.broadcast(user_items_score)
#     /* 
#      * 提取item_sim_user_score为((user,item2),sim * score) 
#      * RDD[(user,item2),sim * score] 
#      */  

    user_item_simscore = user_item_score.flatMap(lambda f:getOrElse(f,user_sim_bd,user_items_score_bd))\
                                        .filter(lambda f:f[1]> 0.03)  
#       /*
#      * 聚合user_item_simscore为 (user,（item2,sim1 * score1 + sim2 * score2）)
#      * 假设user观看过两个item,评分分别为score1和score2，item2是与user观看过的两个item相似的item,相似度分别为sim1，sim2 
#      * RDD[(user,item2),sim1 * score1 + sim2 * score2）)] 
#      */  
    user_item_rank = user_item_simscore.reduceByKey(add,1000)  

#     /* 
#      * 过滤用户已看过的item,并对user_item_rank基于user聚合 
#      * RDD[(user,CompactBuffer((item2,rank2）,(item3,rank3)...))] 
#      */  
    user_items_ranks = user_item_rank.subtractByKey(user_item_score)\
                                     .map(lambda f:(f[0][0], (f[0][1], f[1])))\
                                     .groupByKey()  
#     /* 
#      * 对user_items_ranks基于rank降序排序，并提取topN,其中包括用户已观看过的item 
#      * RDD[(user,ArrayBuffer((item2,rank2）,...,(itemN,rankN)))] 
#      */  
    user_items_ranks_desc = user_items_ranks.mapValues(list)\
                            .mapValues(lambda x:sorted(x,key=itemgetter(1),reverse=True)[:topN])
    return user_items_ranks_desc

from Evaluator import Evaluator
def evaluate(evaluator):
    precise = evaluator.precision()
    coverage = evaluator.coverage()
    popularity = evaluator.popularity()
    recall = evaluator.recall()
    return precise,recall,coverage,popularity

if __name__ == "__main__": 
    import os
    PYSPARK_PYTHON = "/usr/bin/python2.7"
    os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
    conf = SparkConf().setAppName("itemCF").setMaster("yarn")
    conf.set('spark.yarn.dist.files',
            'file:/root/hadoop-2.6/spark/python/lib/pyspark.zip,file:/root/hadoop-2.6/spark/python/lib/py4j-0.10.4-src.zip')
    conf.set('spark.executor.cores','30')
    conf.set('spark.executor.memory','95g')
    conf.set('spark.executor.instances','8')
    conf.set('spark.sql.shuffle.partitions','400')
    conf.set('spark.default.parallelism','200')
    conf.set('spark.scheduler.mode','fair')
#     conf.set("spark.shuffle.file.buffer","128k").set("spark.reducer.maxSizeInFlight","96M")
    spark = SparkSession.builder\
        .config(conf=conf) \
        .getOrCreate()
    sc=spark.sparkContext 
    sc.setLogLevel('WARN')
    start = time.time()
    inputPath = 'data/ml-1m/ratings.dat'
    schema = StructType([
            StructField("user", StringType(), True),
            StructField("item", StringType(), True),
            StructField("rating", DoubleType(), True),
            StructField("timestamp", LongType(), True)])
    ratingRdd = sc.textFile(inputPath).map(lambda line:line.split("::"))\
                    .map(lambda x:(x[0],x[1],float(x[2]),long(x[3])))
    ratingDf = spark.createDataFrame(data=ratingRdd,schema=schema)
#     ratingDf,_ = ratingDf.randomSplit([1.0,9.0],seed=40)
    ratingDf = ratingDf.repartition(300)
    ratingDf.printSchema()
    ratingDf.show(5)
    n = ratingDf.count()
#     ratingDf = ratingDf.withColumn('score',col('rating')*0+1).select('user','item','score')    
    print 'total lines: %s' %n
    train,test = ratingDf.randomSplit([4.0,1.0],seed=40)
    train.cache()
    test.cache()
    userCF = UserCFModel(df=train)
    userCF.train()
    for NN in [5,10,20,40,60,80,100,120]:
        recTopN = recommend(train,userCF.user_pair_sim,NN=NN,topN=10,Normalization=False)
        pre = spark.createDataFrame(data=recTopN.flatMapValues(lambda x:x).map(lambda x:(x[0],x[1][0],x[1][1])),
                                    schema=['user','item','rating'])
        evaluator = Evaluator(train,test,pre)
        (precise,recall,coverage,popularity) = evaluate(evaluator)
        end = time.time()
        print ('NN:%3d,  precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f, time: %s s'
                %(NN,precise*100,recall*100,coverage*100,popularity,end-start))
        pre.write.parquet(partitionBy='user',path='data/ml-1m/result/userCF/uscf_'+str(NN),mode='ignore')
        start = time.time()

root
 |-- user: string (nullable = true)
 |-- item: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)

+----+----+------+---------+
|user|item|rating|timestamp|
+----+----+------+---------+
|3093|   2|   3.0|969650197|
|3095|2174|   4.0|970774487|
|3097|2412|   1.0|969637398|
|3100|3005|   3.0|969591786|
|3101|1210|   4.0|969582345|
+----+----+------+---------+
only showing top 5 rows

total lines: 1000209
NN:  5,  precise:25.42%,  recall:7.68%,  coverage:44.22%,  popularity:6.88, time: 1336.73878813 s


In [27]:
##### !/usr/bin/env python
#-*- coding:utf-8 -*-
############################
#File Name: recommendator.py
#Author: yanbin
#Mail: yanbin918@gmail.com
#Created Time: 2017-08-17 10:
'''
inputPath : 文件数据路径
split_char ： 文件数据每行分隔符（分隔后取前两个作为用户和物品）
读取文件生成dataset类
dataset.user_item_dict：{user1:{item1:1,item2:1},user2:{item1:1}}
dataset.user_item_matrix：M[user][item]=1,if user对item有行为 else M[user][item]=0
dataset.userIndexer,dataset.itemIndexer. dataset索引器，作用通过字符ID 找到 矩阵索引
result_pt ： 预测结果存入路径
userCF_prediction_result_pt ：userCF预测结果保存变量名
itemCF_prediction_result_pt ：itemCF预测结果保存变量名
random_prediction_result_pt ：random预测结果保存变量名
topn_prediction_result_pt ：topn预测结果保存变量名
lfm_prediction_result_pt ：lfm预测结果保存变量名
'''
import sys
sys.path.append('../')
from ml_latest_small.data import *
from Collaborative_Filter.itemBased import ItemCFModel,ItemCF
from Collaborative_Filter.userBased import UserCFModel,UserCF
from Collaborative_Filter.lfmBased import lfmModel,lfm
from Comparision.randomBased import RandomModel
from Comparision.topnBased import topnModel
import pickle
import multiprocessing
import os
from Evaluation.Evaluator import Evaluator
import time
def evaluate(evaluator):
    precise = evaluator.precision()
    coverage = evaluator.coverage()
    popularity = evaluator.popularity()
    recall = evaluator.recall()
    return precise,recall,coverage,popularity

def userCF_predict(train_dict,test_dict,UserCF_model,K):

    #recommendation for every user in train_Dataset
    users = train_dict.keys()
    user_nums = len(users)
    i = 0
    prediction_dict ={}
    for user in users:
        prediction_dict[user]=UserCF_model.recommendation(user,config={'K':K,'N':10}).keys()
    return prediction_dict
start = time.time()
train_dict = train.rdd.map(lambda x:(x['user'],(x['item'],x['rating'])))\
             .groupByKey().mapValues(dict).collectAsMap()
test_dict = test.rdd.map(lambda x:(x['user'],(x['item'],x['rating'])))\
             .groupByKey().mapValues(dict).collectAsMap()
userCF = UserCF()
#train UserCF_model
print "训练用户基于物品协同过滤算法模型........"
UserCF_model=userCF.train(train_dict)
#train UserCF_model
for K in [5,10,20,40,60,80,100,120]:
    prediction_dict = userCF_predict(train_dict,test_dict,UserCF_model,K) 
    evaluator = Evaluator(train_dict,test_dict,prediction_dict,N=10)
    (precise,recall,coverage,popularity) = evaluate(evaluator)
    end = time.time()
    print ('NN:%3d,  precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f, time: %s s'
            %(K,precise*100,recall*100,coverage*100,popularity,end-start))
    start = time.time()    

训练用户基于物品协同过滤算法模型........
NN:  5,  precise:0.26%,  recall:0.77%,  coverage:64.48%,  popularity:4.18, time: 145.216445923 s
NN: 10,  precise:0.43%,  recall:1.29%,  coverage:65.95%,  popularity:4.25, time: 39.5411481857 s
NN: 20,  precise:0.56%,  recall:1.67%,  coverage:59.39%,  popularity:4.38, time: 40.4920840263 s


KeyboardInterrupt: 